# Setup

In [ ]:
# Parameters
videos_dir = None
scene_json_dirs = None
save_dir_all = None
# Change this to the desired number of frames per segment
num_frames_per_segment = None

In [ ]:
import os

dir_path = os.getcwd()


if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
    
if not scene_json_dirs:
    scene_json_dirs = f'{dir_path}/SceneJSON'
    
if not save_dir_all:
    save_dir_all = f'{dir_path}/Keyframes'
    
if not num_frames_per_segment:
    num_frames_per_segment = 5  # Change this to the desired number of frames per segment

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

fatal: destination path 'TransNetV2' already exists and is not an empty directory.


Updated Git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/feat/metadata


In [ ]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from TransNetV2.inference.transnetv2 import TransNetV2

2024-07-28 12:35:38.861313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 12:35:39.050740: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 12:35:40.076194: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Process cutting frames

In [ ]:
def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = f'{videos_dir}/Videos_{data_part}/video'
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split(
            '_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = f'{data_part_path}/{video_path}'
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths


def get_evenly_spaced_frames(start_idx, end_idx, num_frames):
    """Get evenly spaced frame indices between start_idx and end_idx."""
    return np.linspace(start_idx, end_idx, num_frames).astype(int)


def sample_frames(video_path, scene_json_path, save_dir, num_frames_per_segment, width=48, height=27):
    """Sample frames from video segments and save them to the specified directory."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    with open(scene_json_path, 'r') as f:
        video_scenes = json.load(f)

    cap = cv2.VideoCapture(video_path)
    for i, shot in enumerate(tqdm(video_scenes)):
        shot_frames_id = get_evenly_spaced_frames(
            shot[0], shot[1], num_frames_per_segment)
        for index in shot_frames_id:
            cap.set(cv2.CAP_PROP_POS_FRAMES, index)
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (width, height))
                filename = f"{save_dir}/{index:06d}.jpg"
                if not cv2.imwrite(filename, frame):
                    print('Failed to save frame:', filename)
            else:
                print('Failed to read frame at index:', index)
    cap.release()


def process_videos(all_video_paths, scene_json_dirs, save_dir_all, num_frames_per_segment):
    """Process all videos and sample frames from their scenes."""
    if not os.path.exists(save_dir_all):
        os.mkdir(save_dir_all)

    for key in all_video_paths.keys():
        save_dir = f'{save_dir_all}/{key}_extra'

        if not os.path.exists(save_dir):
            os.mkdir(save_dir)

        video_paths_dict = all_video_paths[key]
        video_ids = sorted(video_paths_dict.keys())
        for video_id in tqdm(video_ids):
            video_path = video_paths_dict[video_id]
            video_scene_path = f'{scene_json_dirs}/{key}/{video_id}.json'

            save_dir_video = f'{save_dir}/{video_id}'
            sample_frames(video_path, video_scene_path,
                          save_dir_video, num_frames_per_segment)

In [ ]:
all_video_paths = parse_video_info(videos_dir)
process_videos(all_video_paths, scene_json_dirs, save_dir_all, num_frames_per_segment)

  0%|          | 0/30 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/SceneJSON/L21/V001.json'